In [ ]:
import requests
import zipfile
import os
import json

# URL du fichier ZIP
url = "https://opendata.paris.fr/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/"

# Nom local du fichier ZIP
zip_filename = "comptages_routiers_2020.zip"

# Dossier où extraire les fichiers
output_folder = "extracted_data"

# Nombre de lignes à lire par fichier
max_lines = 1000

# Étape 1 : Télécharger le fichier ZIP
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(zip_filename, "wb") as f:
        f.write(response.content)
    print(f"Fichier téléchargé : {zip_filename}")
else:
    print(f"Erreur lors du téléchargement : {response.status_code}")
    exit()

# Étape 2 : Extraire le contenu du fichier ZIP (limité à deux fichiers)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    all_files = zip_ref.namelist()[:2]  # Limiter à 2 fichiers
    for file in all_files:
        zip_ref.extract(file, output_folder)
    print(f"Deux premiers fichiers extraits : {all_files}")

# Étape 3 : Lire partiellement les fichiers texte et les convertir en JSON
extracted_files = os.listdir(output_folder)
converted_files = []

for file in extracted_files:
    file_path = os.path.join(output_folder, file)
    if file.endswith(".txt"):  # Assurez-vous que le fichier est un texte
        try:
            # Lire les premières lignes seulement
            with open(file_path, "r", encoding="utf-8") as f:
                lines = [next(f).strip() for _ in range(max_lines) if not f.tell() == os.fstat(f.fileno()).st_size]

            # Convertir en JSON
            json_data = [line.split("\t") for line in lines


In [ ]:
import json

# Exemple de grille (adapter aux limites de Paris)
grille = [
    {"id": 1, "lat_min": 48.85, "lat_max": 48.86, "lon_min": 2.29, "lon_max": 2.30},
    {"id": 2, "lat_min": 48.86, "lat_max": 48.87, "lon_min": 2.30, "lon_max": 2.31},
    # Ajouter autant de carrés nécessaires
]

# Fonction pour trouver le carré correspondant à un point
def trouver_carre(lat, lon, grille):
    for carre in grille:
        if carre["lat_min"] <= lat < carre["lat_max"] and carre["lon_min"] <= lon < carre["lon_max"]:
            return carre["id"]
    return None

# Charger les données JSON
with open("data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Initialiser le regroupement
regroupement = {carre["id"]: {"somme_k": 0, "count_k": 0} for carre in grille}

# Parcourir les données pour les regrouper par carré
for item in data["results"]:
    coord = item["geo_point_2d"]
    lat, lon = coord["lat"], coord["lon"]
    valeur_k = item.get("k", None)  # Champ "k" pour la moyenne
    carre_id = trouver_carre(lat, lon, grille)

    if carre_id is not None and valeur_k is not None:
        regroupement[carre_id]["somme_k"] += valeur_k
        regroupement[carre_id]["count_k"] += 1

# Calcul des moyennes pour chaque carré
resultats_final = []
for carre_id, contenu in regroupement.items():
    somme_k = contenu["somme_k"]
    count_k = contenu["count_k"]
    moyenne_k = somme_k / count_k if count_k > 0 else None
    resultats_final.append({"carre_id": carre_id, "moyenne_k": moyenne_k})

# Sauvegarder les résultats dans un fichier JSON
with open("moyennes_par_carre.json", "w", encoding="utf-8") as f:
    json.dump(resultats_final, f, indent=4, ensure_ascii=False)

print("Moyennes calculées et sauvegardées dans 'moyennes_par_carre.json'")
